In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import os

In [3]:
pip install ensemble_boxes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# Load the first object detection model
PATH_TO_SAVED_MODEL1="/content/gdrive/MyDrive/customTF2/data/inference_graph/saved_model"
model1 = tf.saved_model.load(PATH_TO_SAVED_MODEL1)

In [6]:
# Load the first object detection model
PATH_TO_SAVED_MODEL2="/content/gdrive/MyDrive/fasterTF2/data/inference_graph/saved_model"
model2 = tf.saved_model.load(PATH_TO_SAVED_MODEL2)

In [7]:
# set the directory path
dir_path = '/content/gdrive/MyDrive/customTF2/test_images/'
labels = ['Handgun', 'Knife', 'RazorBlade', 'Shuriken']

In [8]:
def savevalues(boxes, scores, classes, savepath, H, W, image_path):
  detections = []
  th = 0.5
  # Loop over all detections and draw detection box if confidence is above minimum threshold
  for i in range(len(scores)):
    if ((scores[i] > th) and (scores[i] <= 1.0)):
      # Get bounding box coordinates and draw box
      # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
      ymin = int(max(1,(boxes[i][0] * H)))
      xmin = int(max(1,(boxes[i][1] * W)))
      ymax = int(min(H,(boxes[i][2] * H)))
      xmax = int(min(W,(boxes[i][3] * W)))

      # Draw label
      object_name = labels[int(classes[i])-1] # Look up object name from "labels" array using class index
      detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])

  # Get filenames and paths
  image_fn = os.path.basename(image_path)      
  base_fn, ext = os.path.splitext(image_fn)
  txt_result_fn = base_fn +'.txt'
  txt_savepath = os.path.join(savepath, txt_result_fn)

  # Write results to text file
  # (Using format defined by https://github.com/Cartucho/mAP, which will make it easy to calculate mAP)
  with open(txt_savepath,'w') as f:
      for detection in detections:
          f.write('%s %.4f %d %d %d %d\n' % (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

In [9]:
# loop through all the files in the directory
for filename in os.listdir(dir_path):
    # check if the file is an image
    if filename.endswith('.png'):
        # open the image file using PIL
        image_path = os.path.join(dir_path, filename)
        image = cv2.imread(image_path)
        image_np = np.array(image)

        # Create the input tensor
        input_tensor = tf.convert_to_tensor(image_np)
        input_tensor = tf.expand_dims(input_tensor, axis=0)
        input_tensor = tf.image.convert_image_dtype(input_tensor, tf.uint8)

        # Make predictions using the first model
        outputs1 = model1(input_tensor)
        boxes1 = outputs1['detection_boxes'].numpy()[0]
        scores1 = outputs1['detection_scores'].numpy()[0]
        classes1 = outputs1['detection_classes'].numpy()[0].astype(np.uint8)

        # Make predictions using the second model
        outputs2 = model2(input_tensor)
        boxes2 = outputs2['detection_boxes'].numpy()[0]
        scores2 = outputs2['detection_scores'].numpy()[0]
        classes2 = outputs2['detection_classes'].numpy()[0].astype(np.uint8)

        b1 = []
        s1 = []
        c1 = []
        th = 0.5
        for i in range(len(scores1)):
          if scores1[i]>th:
            b1.append(boxes1[i])
            s1.append(scores1[i])
            c1.append(classes1[i])
        
        b2 = []
        s2 = []
        c2 = []
        for i in range(len(scores2)):
          if scores2[i]>th:
            b2.append(boxes2[i])
            s2.append(scores2[i])
            c2.append(classes2[i])
        
        from ensemble_boxes import *
        boxes_list = [b1, b2]
        scores_list = [s1, s2]
        labels_list = [c1, c2]
        #weights = [2, 1]

        iou_thr = 0.5
        skip_box_thr = 0.0001
        sigma = 0.1

        boxes1, scores1, classes1 = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        boxes2, scores2, classes2 = soft_nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
        boxes3, scores3, classes3 = non_maximum_weighted(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        boxes4, scores4, classes4 = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)

        H, W, _ = image.shape
        savevalues(boxes1, scores1, classes1, '/content/gdrive/MyDrive/Ensemble/m1/', H, W, image_path)
        savevalues(boxes2, scores2, classes2, '/content/gdrive/MyDrive/Ensemble/m2/', H, W, image_path)
        savevalues(boxes3, scores3, classes3, '/content/gdrive/MyDrive/Ensemble/m3/', H, W, image_path)
        savevalues(boxes4, scores4, classes4, '/content/gdrive/MyDrive/Ensemble/m4/', H, W, image_path)

In [1]:
l1 = [1,2,3]
l2 = [4,5,6]
l = [l1,l2]
l

[[1, 2, 3], [4, 5, 6]]